## First cell calls API and sets up the dataframe we need for the stacked bar chart.
## It works just fine. It takes 2-3 min to run.

In [1]:
# Setup.
import pandas as pd
import requests
import json
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import itertools 
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import matplotlib

def get_race_result(season_number, year ):
    api = 'http://ergast.com/api/f1/{}/{}/qualifying.json'.format(year, season_number)
    data = requests.get(api).json()
    temp_df =  pd.DataFrame.from_records(data.get('MRData').get('RaceTable').get('Races')[0].get('QualifyingResults'))
    temp_df['seasonNbr'] = season_number
    temp_df['year'] = year
    return temp_df

results = []
seasons = np.arange(2003,2018,1) #2018
rounds = np.arange(1,15,1) #15

for s in seasons:
    for r in rounds:
        result = get_race_result(r, s)
        results.append(result)

qualifying = pd.concat(results, axis=0)
qualifying['driver_id'] = qualifying.Driver.map(lambda x: x.get('driverId'))
qualifying['car_make'] = qualifying.Constructor.map(lambda x: x.get('name'))
qualifying['driver_nat'] = qualifying.Driver.map(lambda x: x.get('nationality'))

results = []
seasons = np.arange(2003,2018,1) #2018
rounds = np.arange(1,15,1) #15

for s in seasons:
    for r in rounds:
        result = get_race_result(r, s)
        results.append(result)
        
def get_race_result(season_number, year ):
    api = 'http://ergast.com/api/f1/{}/{}/results.json'.format(year, season_number)
    data = requests.get(api).json()
                                         #resp.get('MRData').get('RaceTable').get('Races')[0].get('Results')
    temp_df =  pd.DataFrame.from_records(data.get('MRData').get('RaceTable').get('Races')[0].get('Results'))
    circuit=data.get('MRData').get('RaceTable').get('Races')[0].get('Circuit')
    temp_df['circuit_name']=circuit.get('circuitName')
    temp_df['seasonNbr'] = season_number
    temp_df['year'] = year
    return temp_df

results = []
seasons = np.arange(2003,2018,1) #2018
rounds = np.arange(1,15,1) #15

for s in seasons:
    for r in rounds:
        result = get_race_result(r, s)
        results.append(result)

out = pd.concat(results, axis=0)
out['driver_id'] = out.Driver.map(lambda x: x.get('driverId'))
out['car_make'] = out.Constructor.map(lambda x: x.get('name'))
out['driver_nat'] = out.Driver.map(lambda x: x.get('nationality'))

results = out
results['points'] = results['points'].astype(float)

# Create combined dataframe of car make, year, position, qualifying counts and results counts.
# Summarize qualifying race results by car, year, and position.
qualifying_car_makes = qualifying.groupby(['car_make','year','position']).size().reset_index()
qualifying_car_makes.rename(columns={0:'qualifying_count'}, inplace = True)

# Summarize race results by car, year, and position.
results_car_makes = results.groupby(['car_make','year','position']).size().reset_index()
results_car_makes.rename(columns={0:'results_count'}, inplace = True)

# Combine the qualifying results counts and race results counts together.
car_make_df = pd.merge(qualifying_car_makes, results_car_makes, on = ['car_make','year','position'])
car_make_df['total_count'] = car_make_df['qualifying_count'] + car_make_df['results_count']

#  Create subset of dataframe for purposes of stacked bar chart.
df2 = car_make_df[['car_make','results_count','position']].groupby(['car_make','position']).sum().reset_index()

df2.head()

,car_make,position,results_count
0,BAR,10,4
1,BAR,11,2
2,BAR,12,3
3,BAR,13,2
4,BAR,14,4


## Here we set up the function to multiply every `car_make` column by  `results_counts` .

In [4]:
def mult(row):
    row["results_count"]*row[col]

df3 = pd.get_dummies(df2,columns=["car_make"])
df3 = df3.apply(pd.to_numeric)

# FOR TUTOR ZOOM SESSION - Why do the value change if I add this line: 
#df3 = df3.drop(['results_count'], axis=1)
#df3 = df3.loc[:, df3.columns != 'results_count')

df3[10:25]

,position,results_count,car_make_BAR,car_make_BMW Sauber,car_make_Brawn,car_make_Caterham,car_make_Ferrari,car_make_Force India,car_make_HRT,car_make_Haas F1 Team,...,car_make_Minardi,car_make_Red Bull,car_make_Renault,car_make_Sauber,car_make_Spyker,car_make_Super Aguri,car_make_Toro Rosso,car_make_Toyota,car_make_Virgin,car_make_Williams
10,20,6,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,3,6,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,4,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,5,7,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,6,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,7,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,8,6,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,9,7,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,10,4,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Why does it return none?!
I thought it was because the car make columns were type `object`, but after converting
them to numeric above, they still fail.

In [5]:
for col in list(df3.columns)[2:]:
    df3[col] = df3.apply(mult,axis=1)
df3.head()

,position,results_count,car_make_BAR,car_make_BMW Sauber,car_make_Brawn,car_make_Caterham,car_make_Ferrari,car_make_Force India,car_make_HRT,car_make_Haas F1 Team,...,car_make_Minardi,car_make_Red Bull,car_make_Renault,car_make_Sauber,car_make_Spyker,car_make_Super Aguri,car_make_Toro Rosso,car_make_Toyota,car_make_Virgin,car_make_Williams
0,10,4,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,11,2,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,12,3,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,13,2,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,14,4,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
